In [4]:
!pip install tensorflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
unstructured-client 0.21.1 requires typing-extensions>=4.7.1, but you have typing-extensions 4.5.0 which is incompatible.
panel 0.14.3 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.2 which is incompatible.
openai 1.12.0 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.
langchain 0.0.228 requires dataclasses-json<0.6.0,>=0.5.7, but you have dataclasses-json 0.6.4 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.


     ------------------------------------ 143.8/143.8 kB 356.3 kB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1


In [73]:
import cv2
import numpy as np
from keras.models import load_model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pandas as pd

def detect_glaucoma(image_path):
    # Read the image with 3 channels (RGB)
    image = cv2.imread(image_path)

    # Check if the image is successfully loaded
    if image is None:
        print(f"Error: Unable to load the image from path: {image_path}")
        return None

    # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(image, (65, 65), 0)

    # Find the pixel with the highest intensity value
    max_intensity_pixel = np.unravel_index(np.argmax(blurred_image), blurred_image.shape)

    # Define the radius for the circle
    radius = 200 // 2

    # Get the x and y coordinates for cropping the image
    x = max_intensity_pixel[1] - radius
    y = max_intensity_pixel[0] - radius

    # Create a mask for the circle
    mask = np.zeros_like(image)
    cv2.circle(mask, (x + radius, y + radius), radius, (255, 255, 255), -1)

    # Apply the mask to the original image
    roi_image = cv2.bitwise_and(image, mask)

    # Split the green channel
    green_channel = roi_image[:, :, 1]
    
    # Apply histogram equalization
    clahe_op = cv2.createCLAHE(clipLimit=2)
    roi_image = clahe_op.apply(green_channel)

    # Save the preprocessed image
    roi_image = cv2.imwrite("image.jpg", roi_image)

    # Load the pre-trained model
    model = load_model("src\detect_glaucoma\GLAUCOMA_DETECTION.h5")
    model.compile(loss='binary_crossentropy',
              optimizer=Adam(1e-4),
              metrics=['binary_accuracy'])

    # Resize the image to (256, 256)
    input_image = cv2.resize(roi_image, (256, 256))
# Create an instance of ImageDataGenerator for preprocessing
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)  # You can add other preprocessing options here

    # Create a DataFrame with the path to the image you want to predict on
    image_path_df = pd.DataFrame({'image_path': ['image.jpg']})

    # Use flow_from_dataframe to generate batches of images for prediction
    # You need to specify the target_size and batch_size
    image_generator = datagen.flow_from_dataframe(
        dataframe=image_path_df,
        x_col='image_path',
        y_col=None,  # For prediction, y_col can be set to None
        target_size=(256, 256),  # Specify the target size of the images
        batch_size=1,  # Set batch_size to 1 for predicting on a single image
        class_mode=None,  # For prediction, class_mode can be set to None
        shuffle=False  # No need to shuffle since there's only one image
    )




    prediction = model.predict(image_generator)

    # Perform further processing or return the prediction result as needed
    return prediction




In [74]:
import tensorflow as tf
import cv2

# Load the input image using OpenCV (replace 'my_image.jpg' with the actual image file path)
cv_img = cv2.imread('my_image.jpg')
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB




error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [75]:
# Example usage
prediction = detect_glaucoma('s;.JPG')
print(prediction)

Found 1 validated image filenames.
1/1 [==============================] - 0s 448ms/step
[[4.233557e-07]]


In [3]:
from utils import process_image, detect_glaucoma
import cv2
from preprocessing_module import custom_preprocessing







# Detect glaucoma in the image
prediction = detect_glaucoma()

if prediction is not None:
    print("Glaucoma detection result:", prediction)
else:
    print("Error occurred while processing the image.")



ValueError: Layer 'conv2d' expected 2 variables, but received 0 variables during loading. Expected: ['conv2d/kernel:0', 'conv2d/bias:0']